MLP modeling
Logistic feature selection
Optuna
Threshold

In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv(r'C:\Users\leeya\OneDrive\Desktop\부캠 수업자료\Final\train_common_v3.3_피처삭제X_군집_1111.csv')
test = pd.read_csv(r'C:\Users\leeya\OneDrive\Desktop\부캠 수업자료\Final\test_common_v3.3_피처삭제X_군집_1111.csv')
target =  pd.read_csv(r'C:\Users\leeya\OneDrive\Desktop\부캠 수업자료\Final\store_train.csv')
submit = pd.read_csv(r'C:\Users\leeya\OneDrive\Desktop\부캠 수업자료\Final\store_submission (2).csv')

In [3]:
train.shape , target.shape , test.shape , submit.shape

((14940, 1471), (14940, 2), (12225, 1471), (12225, 2))

In [4]:
train.isnull().sum().sum(), test.isnull().sum().sum()

(np.int64(0), np.int64(0))

In [5]:
train_ft = train.iloc[:,1:]
test_ft = test.iloc[:,1:]
train_ft.shape, test_ft.shape

((14940, 1470), (12225, 1470))

In [6]:
cols = train_ft.select_dtypes("object").columns.tolist()
train_ft[cols].nunique()

주구매지점        4
주구매_대분류     22
주구매_중분류    238
dtype: int64

In [7]:
from sklearn.preprocessing import OneHotEncoder

cols = train_ft.select_dtypes(include=['object']).columns
enc = OneHotEncoder(handle_unknown="ignore")
enc.fit(train_ft[cols])

tmp = pd.DataFrame(
    enc.transform(train_ft[cols]).toarray(),
    columns = enc.get_feature_names_out()
)
train_ft = pd.concat([train_ft, tmp], axis=1).drop(columns=cols)

tmp = pd.DataFrame(
    enc.transform(test_ft[cols]).toarray(),
    columns = enc.get_feature_names_out()
)
test_ft = pd.concat([test_ft, tmp], axis=1).drop(columns=cols)

In [8]:
train_ft.select_dtypes("object").columns , test_ft.select_dtypes("object").columns

(Index([], dtype='object'), Index([], dtype='object'))

In [9]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
test_ft[test_ft.columns] = scaler.fit_transform(test_ft)
train_ft.head()

,내점일수,구매주기,주말방문비율,평일방문비율,주말방문횟수,평일방문횟수,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,...,주구매_중분류_핸드백행사,주구매_중분류_행사,주구매_중분류_행사슈즈,주구매_중분류_행사핸드백,주구매_중분류_향수,주구매_중분류_헤어ACC,주구매_중분류_홈데코,주구매_중분류_화장잡화,주구매_중분류_화장품,주구매_중분류_훼미닌부틱
0,0.041494,0.130682,0.250000,0.750000,0.032258,0.029014,0.050000,0.250000,0.400000,0.300000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.082988,0.090909,0.023810,0.976190,0.006452,0.079304,0.357143,0.166667,0.357143,0.119048,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.228216,0.034091,0.210526,0.789474,0.154839,0.174081,0.464912,0.140351,0.175439,0.219298,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.377593,0.017045,0.189573,0.810427,0.258065,0.330754,0.379147,0.180095,0.236967,0.203791,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.107884,0.062500,0.258065,0.741935,0.103226,0.088975,0.112903,0.612903,0.209677,0.064516,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
target = target["target"]
target

0        1.0
1        1.0
2        0.0
3        0.0
4        0.0
        ... 
14935    0.0
14936    0.0
14937    0.0
14938    1.0
14939    1.0
Name: target, Length: 14940, dtype: float64

In [11]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(train_ft, target, test_size=0.2, random_state=42)

In [12]:
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import optuna
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score

In [ ]:
SEED = 42
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

# Logistic 피처셀렉션
logistic = LogisticRegression(penalty="l2", C=1.0, solver="liblinear", random_state=SEED)
selector = SelectFromModel(logistic)
selected_features = selector.fit_transform(train_ft, target)


selected_cols = selector.get_feature_names_out()
print(f"Selected features: {len(selected_cols)}")

Selected features: 691


In [ ]:
import optuna
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.neural_network import MLPClassifier

# Optuna 하이퍼파라미터 튜닝
def objective(trial):
    # MLPClassifier 하이퍼파라미터 설정
    n_layers = trial.suggest_int("n_layers", 1, 3)
    hidden_layer_sizes = tuple([trial.suggest_int(f"n_units_l{i}", 10, 100) for i in range(n_layers)])

    hp = {
        "hidden_layer_sizes": hidden_layer_sizes,
        "activation": trial.suggest_categorical("activation", ["relu", "tanh", "logistic"]),
        "solver": trial.suggest_categorical("solver", ["adam", "sgd"]),
        "alpha": trial.suggest_float("alpha", 1e-5, 1e-1, log=True),
        "learning_rate_init": trial.suggest_float("learning_rate_init", 1e-5, 1e-1, log=True),
        "max_iter": trial.suggest_int("max_iter", 100, 500, step=50),
        "early_stopping": True,
        "validation_fraction": trial.suggest_float("validation_fraction", 0.1, 0.3, step=0.05),
        "n_iter_no_change": trial.suggest_int("n_iter_no_change", 5, 15),
    }


    model = MLPClassifier(**hp, random_state=SEED)


    score = cross_val_score(model, selected_features, target, cv=cv, scoring="f1_macro", n_jobs=-1).mean()

    # 성능이 0.72 미만이면 무시 (float("-inf")로 반환하여 이 trial을 최적화 과정에서 제외)
    if score < 0.72:
        return float("-inf")
    
    return score


sampler = optuna.samplers.TPESampler(seed=SEED)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=100)


print("Best hyperparameters:", study.best_params)
print("Best F1 Score:", study.best_value)


[I 2024-11-11 23:18:30,870] A new study created in memory with name: no-name-47cf10d0-b8d4-4460-8654-1354500b0a6b


[I 2024-11-11 23:19:16,797] Trial 0 finished with value: -inf and parameters: {'n_layers': 2, 'n_units_l0': 96, 'n_units_l1': 76, 'activation': 'relu', 'solver': 'sgd', 'alpha': 0.002537815508265664, 'learning_rate_init': 0.006796578090758156, 'max_iter': 100, 'validation_fraction': 0.3, 'n_iter_no_change': 14}. Best is trial 0 with value: -inf.
[I 2024-11-11 23:19:24,645] Trial 1 finished with value: -inf and parameters: {'n_layers': 1, 'n_units_l0': 26, 'activation': 'logistic', 'solver': 'adam', 'alpha': 0.0028016351587162596, 'learning_rate_init': 3.613894271216525e-05, 'max_iter': 200, 'validation_fraction': 0.15000000000000002, 'n_iter_no_change': 10}. Best is trial 0 with value: -inf.
[I 2024-11-11 23:19:36,677] Trial 2 finished with value: 0.7211096991797479 and parameters: {'n_layers': 3, 'n_units_l0': 28, 'n_units_l1': 56, 'n_units_l2': 63, 'activation': 'tanh', 'solver': 'sgd', 'alpha': 0.07286653737491042, 'learning_rate_init': 0.01712337597316398, 'max_iter': 200, 'validat

Best hyperparameters: {'n_layers': 2, 'n_units_l0': 88, 'n_units_l1': 89, 'activation': 'relu', 'solver': 'adam', 'alpha': 0.001230314130334757, 'learning_rate_init': 0.0016991055449377132, 'max_iter': 500, 'validation_fraction': 0.15000000000000002, 'n_iter_no_change': 10}
Best F1 Score: 0.7263444194158325


In [ ]:
# Optuna 튜닝 결과
best_mlp_params = {
    'hidden_layer_sizes': (88, 89), 
    'activation': 'relu',
    'solver': 'adam',
    'alpha': 0.001230314130334757,
    'learning_rate_init': 0.0016991055449377132,
    'max_iter': 500,
    'validation_fraction': 0.15000000000000002,
    'n_iter_no_change': 10,
    'early_stopping': True
}


scores = []
models = []

for tri, vai in cv.split(train_ft, target):
    x_train = train_ft.iloc[tri][selected_cols] #logistic 으로 선택된 피처만 사용가능
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai][selected_cols]  
    y_valid = target.iloc[vai]

    mlp_model = MLPClassifier(**best_mlp_params, random_state=SEED)
    mlp_model.fit(x_train, y_train)

    models.append(mlp_model)

    pred = mlp_model.predict(x_valid)
    score = f1_score(y_valid, pred, average='macro')
    scores.append(score)

#f1 출력
print("Mean F1 Macro Score:", np.mean(scores))


Mean F1 Macro Score: 0.7263444194158325


In [ ]:
#proba 파일

mlp_model = MLPClassifier(**best_mlp_params, random_state=SEED)
mlp_model.fit(train_ft[selected_cols], target)


pred = mlp_model.predict_proba(test_ft[selected_cols])[:, 1]

submit["target"] = pred
submit.to_csv("mlp+optuna+logistic_fs.csv", index=False)

In [24]:
#submit 파일

sub_pred = mlp_model.predict_proba(test_ft[selected_cols])
submit["target"] = sub_pred
submit.to_csv("mlp_optuna_logistic_fs_sub.csv", index=False)

In [ ]:
#임계값 조절 submit
threshold = 0.42
submit["target"] = (sub_pred[:, 1] >= threshold).astype(int)
submit.to_csv("mlp_optuna_logistic_fs_threshold_sub.csv", index=False)


In [21]:
submit.shape

(12225, 2)

In [22]:
submit.head

<bound method NDFrame.head of                ID    target
0          test_0  0.862846
1          test_1  0.778231
2          test_2  0.818966
3          test_3  0.109296
4          test_4  0.562952
...           ...       ...
12220  test_12220  0.313295
12221  test_12221  0.703195
12222  test_12222  0.932361
12223  test_12223  0.261959
12224  test_12224  0.919641

[12225 rows x 2 columns]>